# Preprocessing for Neural Network

## Notebook Summary

- [Notebook Setup](#Notebook-Setup)
- [Read in Parquet](#Read-in-Parquet)
- [Encode Features](#Encode-Features)
- [Balance Target Features](#Balance-Target-Features)
- [Results](#Results)
- [Save as Parquet](#Save-as_Parquet)

## Notebook Setup

In [1]:
from assignment_3_tools import (
    parquet_to_dict, dict_to_parquet, encode_cvd_var,
    rand_under_sample, rand_over_sample, balanced_dict)
import polars as pl
import pandas as pd
import seaborn as sns

## Read in Parquet

In [2]:
pq_jar = parquet_to_dict('../../Data/GoogleDrive/')

## Encode Features

In [3]:
df_heart_drop_05_imp = pq_jar['df_heart_drop_05_imp'].collect().to_pandas()

X_train, X_test, y_train, y_test = encode_cvd_var(df_heart_drop_05_imp)

## Balance Target Features

In [4]:
ratios = [1,2,3,5,7]
balanced_X_trains, balanced_y_trains = balanced_dict(X_train, y_train, ratios)
balanced_X_trains['Baseline'] = X_train
balanced_y_trains['Baseline'] = y_train 

## Results

- [x] Make a table of over/under sampled datasets
- [] Add MLP performance

In [5]:
Balance_Method = list()
No_CVD = list()
Yes_CVD = list()
Total_Obs = list()

for key, value in balanced_y_trains.items():
    Balance_Method.append(key)
    No_CVD.append(value.value_counts()[0])
    Yes_CVD.append(value.value_counts()[1])
    Total_Obs.append(value.value_counts()[1] + value.value_counts()[0])

tbl_balanced = pd.DataFrame({
    'Balance_Method': Balance_Method,
    'No_CVD': No_CVD,
    'Yes_CVD': Yes_CVD,
    'Total_Obs': Total_Obs})

tbl_balanced.sort_values(by='Balance_Method', ascending=True)

,Balance_Method,No_CVD,Yes_CVD,Total_Obs
10,Baseline,260572,35453,296025
1,Over_Sample_1:1,260572,260572,521144
3,Over_Sample_1:2,521144,260572,781716
5,Over_Sample_1:3,781716,260572,1042288
7,Over_Sample_1:5,1302860,260572,1563432
9,Over_Sample_1:7,1824004,260572,2084576
0,Under_Sample_1:1,35453,35453,70906
2,Under_Sample_2:1,70906,35453,106359
4,Under_Sample_3:1,106359,35453,141812
6,Under_Sample_5:1,177265,35453,212718


## Save as Parquet